In [1]:
import pandas as pd
import io
from google.cloud import storage
# Create a client
client = storage.Client()

# Define your bucket name
bucket_name = "dsgt-clef-pan-2024"
# Get the bucket
bucket = client.get_bucket(bucket_name)

# List objects in the bucket
blobs = bucket.list_blobs()
for blob in blobs:
    print(blob.name)

PAN Data/Artem.pkl
PAN Data/GPT-wiki-intro.csv
PAN Data/GPT.pkl
PAN Data/HFD.pkl
PAN Data/HF_NabeelShar_ai_and_human_text.csv
PAN Data/HF_artem9k_ai_text_detection_pile/train-00000-of-00007-bc5952582e004d67.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00001-of-00007-71c80017bc45f30d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00002-of-00007-ee2d43f396e78fbc.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00003-of-00007-529931154b42b51d.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00004-of-00007-b269dc49374a2c0b.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00005-of-00007-3dce5e05ddbad789.parquet
PAN Data/HF_artem9k_ai_text_detection_pile/train-00006-of-00007-3d8a471ba0cf1c8d.parquet
PAN Data/HF_dmitva_human_ai_generated_text.csv
PAN Data/JackGraphData/
PAN Data/Nabeel.pkl
PAN Data/PAN Competition.zip


In [2]:
# Access a specific blob (file) and read its content
blob = bucket.get_blob("PAN Data/GPT.pkl")
content = blob.download_as_string()

# Load CSV content into a pandas DataFrame
#df = pd.read_csv(io.BytesIO(content))
data = pd.read_pickle(io.BytesIO(content)).reset_index(drop=True)

In [3]:
print(data)

# Define the destination path and filename in the bucket
#destination_blob_name = "path/in/bucket/data.csv"

# Get the bucket
#bucket = client.get_bucket(bucket_name)

# Create a blob (file) in the bucket
#blob = bucket.blob(destination_blob_name)

# Upload the CSV data to the blob
#blob.upload_from_string(csv_data, content_type='text/csv')
#blob.upload_from_string(pickle_data, content_type='application/octet-stream')

                                                     text source Type
0       sexhow railway station was a railway station l...     ai  GPT
1       in finnish folklore, all places and things, an...     ai  GPT
2       in mathematics, specifically differential calc...     ai  GPT
3       is a japanese shōjo manga series written and i...     ai  GPT
4       robert milner "rob" bradley, jr. (born august ...     ai  GPT
...                                                   ...    ...  ...
299995  randy borum is a professor and coordinator of ...  human  GPT
299996  sa'och (, also, "sauch") is an endangered, nea...  human  GPT
299997  philip c. hanawalt (born 1931) is an american ...  human  GPT
299998  vossius gymnasium is a public gymnasium in ams...  human  GPT
299999  simone stratigo (, symeon filippos stratigos; ...  human  GPT

[300000 rows x 3 columns]


In [4]:
#import pandas as pd

#data = pd.read_pickle("/home/abindal/pan-2024/PAN/GPT.pkl")

#print(data.head())

In [5]:
from datasets import load_dataset, Dataset, DatasetDict

# In data map source to 0 or 1 based on ai or human
def map_source(source):
    if source == 'ai':
        return 1
    else:
        return 0
data['label'] = data['source'].apply(map_source)

# Drop source and Type
data = data.drop(['source', 'Type'], axis=1) 

df = Dataset.from_pandas(data)
print(df)

/home/abindal/pan-2024/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['text', 'label'],
    num_rows: 300000
})


In [6]:
print(df[0])

{'text': 'sexhow railway station was a railway station located in the town of sexhow, on the cumbrian coast line in north west england. the station was opened by the lancashire and yorkshire railway on 7 october 1870. it was closed to passengers on 5 january 1950, and to goods on 12 may 1965. \n\nthe station building is now a private residence. there is a small amount of trackage remaining near the building, used currently by a local agricultural business. located in the town of sexhow, on the cumbrian coas', 'label': 1}


## Split Dataset

In [7]:
trainTestValid = df.train_test_split(test_size=0.2, shuffle=True)
testValid = trainTestValid['test'].train_test_split(test_size=0.5, shuffle=True)
trainTestValid_dataset = DatasetDict({
    'train': trainTestValid['train'], 
    'test': testValid['test'], 
    'valid': testValid['train']})

In [8]:
print(trainTestValid_dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 240000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 30000
    })
})


# Preprocess

In [9]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = trainTestValid_dataset.map(tokenize_function, batched=True)

Map: 100%|██████████| 30000/30000 [00:08<00:00, 3566.60 examples/s]


In [10]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=1, lora_alpha=1, lora_dropout=0.1
)

In [11]:
# Get roberta model for classification from Hugging Face
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [13]:
# Print trainable parameters
print(model.print_trainable_parameters())

trainable params: 628,994 || all params: 125,276,164 || trainable%: 0.5020859355176297
None


In [23]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="train/LoRA/", 
    evaluation_strategy="epoch",
    num_train_epochs=20,
    learning_rate=1e-3,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    weight_decay=0.01,
    save_strategy="epoch",
    load_best_model_at_end=True,
    )

In [15]:
!pip install evaluate
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 240000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 30000
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 30000
    })
})


In [24]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = tokenized_datasets['train'].shuffle(seed=42).select(range(5000)),
    eval_dataset = tokenized_datasets['valid'].shuffle(seed=42).select(range(5000)),
    compute_metrics=compute_metrics,
)

In [25]:
trainer.train()

TrainOutput(global_step=4180, training_loss=0.013448994227408757, metrics={'train_runtime': 10219.7319, 'train_samples_per_second': 9.785, 'train_steps_per_second': 0.409, 'total_flos': 2.65043324928e+16, 'train_loss': 0.013448994227408757, 'epoch': 20.0})

In [26]:
model.save_pretrained("train/LoRA/")

In [27]:
# Run inference on the test dataset
predictions = trainer.predict(tokenized_datasets['test'].shuffle(seed=42))
print(predictions)
#Check the accuracy of predictions
print(compute_metrics(predictions))

PredictionOutput(predictions=array([[ 4.296029  , -4.6336985 ],
       [-7.144358  ,  7.8646517 ],
       [-7.199089  ,  7.899159  ],
       ...,
       [-0.37958756,  0.43389043],
       [ 0.20117833, -0.16065288],
       [ 3.0410523 , -3.275156  ]], dtype=float32), label_ids=array([0, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.044846583157777786, 'test_accuracy': 0.9863666666666666, 'test_runtime': 934.0487, 'test_samples_per_second': 32.118, 'test_steps_per_second': 1.338})


ValueError: too many values to unpack (expected 2)

In [31]:
#Print test accuracy
print(predictions)

PredictionOutput(predictions=array([[ 4.296029  , -4.6336985 ],
       [-7.144358  ,  7.8646517 ],
       [-7.199089  ,  7.899159  ],
       ...,
       [-0.37958756,  0.43389043],
       [ 0.20117833, -0.16065288],
       [ 3.0410523 , -3.275156  ]], dtype=float32), label_ids=array([0, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.044846583157777786, 'test_accuracy': 0.9863666666666666, 'test_runtime': 934.0487, 'test_samples_per_second': 32.118, 'test_steps_per_second': 1.338})


In [36]:
print(predictions.metrics)

{'test_loss': 0.044846583157777786, 'test_accuracy': 0.9863666666666666, 'test_runtime': 934.0487, 'test_samples_per_second': 32.118, 'test_steps_per_second': 1.338}
